In [1]:
!git clone https://github.com/androidwoman/stock_time_llm.git
%cd /content/stock_time_llm

Cloning into 'stock_time_llm'...
remote: Enumerating objects: 67, done.
remote: Counting objects: 100% (67/67), done.
remote: Compressing objects: 100% (64/64), done.
remote: Total 67 (delta 19), reused 0 (delta 0), pack-reused 0 (from 0)
Receiving objects: 100% (67/67), 1.04 MiB | 2.82 MiB/s, done.
Resolving deltas: 100% (19/19), done.
/content/stock_time_llm


In [2]:
pip install -r requirements.txt


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.4/60.4 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.0/57.0 kB 3.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.9/116.9 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 32.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 755.5/755.5 MB 2.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 290.1/290.1 kB 14.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.6/44.6 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.6/11.6 MB 28.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.1/17.1 MB 34.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.1/12.1 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.6/9.6 MB 44.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 38.4/38.4 MB 11.4 MB/s eta 0:00:00
   ━━━

In [4]:
import yfinance as yf
import pandas as pd

# Download stock data for a specific ticker, e.g., Apple (AAPL)
data = yf.download('AAPL', start='2000-01-01', end='2024-01-01', interval='1d')
data.reset_index(inplace=True)

# Prepare the data by selecting necessary columns and renaming them
data = data[['Date', 'Open', 'High', 'Low', 'Close', 'Volume']]
data.columns = ['timestamp', 'open', 'high', 'low', 'close', 'volume']

# Save the data to CSV if needed
data.to_csv('AAPL_stock_data.csv', index=False)


[*********************100%***********************]  1 of 1 completed


In [5]:
CHECKPOINT_DIR="./checkpoints/AAPL"

# Ensure the checkpoint directory exists
!mkdir -p $CHECKPOINT_DIR

In [ ]:
ls

AAPL_stock_data.csv  checkpoints/  sample_data/  stock_time_llm/


In [6]:
import pandas as pd

df = data
df.rename(columns={'timestamp': 'date'}, inplace=True)
df.rename(columns={'close': 'y'}, inplace=True)
df = df[['date', 'y']]
df.to_csv('/content/stock_time_llm/dataset/AAPL_stock_data.csv', index=False)


<ipython-input-6-716a47fc6cce>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'timestamp': 'date'}, inplace=True)
<ipython-input-6-716a47fc6cce>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.rename(columns={'close': 'y'}, inplace=True)


In [7]:
df

,date,y
0,2000-01-03,0.999442
1,2000-01-04,0.915179
2,2000-01-05,0.928571
3,2000-01-06,0.848214
4,2000-01-07,0.888393
...,...,...
6032,2023-12-22,193.600006
6033,2023-12-26,193.050003
6034,2023-12-27,193.149994
6035,2023-12-28,193.580002


In [2]:
%cd stock_time_llm

/content/stock_time_llm


In [10]:
!python run_main.py \
    --task_name long_term_forecast \
    --is_training 1 \
    --model_id AAPL_TimeLLM \
    --model TimeLLM \
    --data_path 'AAPL_stock_data.csv' \
    --root_path './dataset/' \
    --model_comment "Stock price prediction" \
    --target 'y' \
    --freq 'd' \
    --seq_len 64 \
    --label_len 64 \
    --pred_len 1 \
    --train_epochs 5 \
    --batch_size 16 \
    --learning_rate 0.001 \
    --patience 5 \
    --checkpoints './checkpoints/AAPL' \
    --loss MSE \
    --lradj type1 \
    --prompt_domain 1 \
    --data Traffic


-------------------------------
0
['2000-01-03T00:00:00.000000000' '2000-01-04T00:00:00.000000000'
 '2000-01-05T00:00:00.000000000' ... '2020-05-21T00:00:00.000000000'
 '2020-05-22T00:00:00.000000000' '2020-05-26T00:00:00.000000000']
-------------------------------
1
['2020-02-25T00:00:00.000000000' '2020-02-26T00:00:00.000000000'
 '2020-02-27T00:00:00.000000000' '2020-02-28T00:00:00.000000000'
 '2020-03-02T00:00:00.000000000' '2020-03-03T00:00:00.000000000'
 '2020-03-04T00:00:00.000000000' '2020-03-05T00:00:00.000000000'
 '2020-03-06T00:00:00.000000000' '2020-03-09T00:00:00.000000000'
 '2020-03-10T00:00:00.000000000' '2020-03-11T00:00:00.000000000'
 '2020-03-12T00:00:00.000000000' '2020-03-13T00:00:00.000000000'
 '2020-03-16T00:00:00.000000000' '2020-03-17T00:00:00.000000000'
 '2020-03-18T00:00:00.000000000' '2020-03-19T00:00:00.000000000'
 '2020-03-20T00:00:00.000000000' '2020-03-23T00:00:00.000000000'
 '2020-03-24T00:00:00.000000000' '2020-03-25T00:00:00.000000000'
 '2020-03-26T00:0

In [12]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [9]:
!pip install mpi4py


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 464.8/464.8 kB 9.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for mpi4py: filename=mpi4py-4.0.0-cp310-cp310-linux_x86_64.whl size=4266271 sha256=51debe4cd40daa3dc737a9665e14ef217aad06d2f47b7286550368761ef1bd87
  Stored in directory: /root/.cache/pip/wheels/96/17/12/83db63ee0ae5c4b040ee87f2e5c813aea4728b55ec6a37317c
Successfully built mpi4py
